In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers

2023-04-28 09:31:25.948760: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-28 09:31:25.948881: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Import the data

In [2]:
training = pd.read_csv("training over and under 3.csv")
test = pd.read_csv("val3.csv")

training = training[["Bentuk_pemanfaatan","Peruntukan","Pusat_kota","Visibilitas","Bangunan","Luas"]]
test = test[["Bentuk_pemanfaatan","Peruntukan","Pusat_kota","Visibilitas","Bangunan","Luas"]]

# Change categorical to number 

In [3]:
condition = [  training.Pusat_kota == "Pusat",
             training.Pusat_kota == "Sedang",
              training.Pusat_kota == "Pinggir",
]

value = [3,2,1]

training.Pusat_kota = np.select(condition,value)
####
####

condition = [  training.Visibilitas == "Strategis",
             training.Visibilitas == "Sedang",
              training.Visibilitas == "Kurang",
]

value = [3,2,1]

training.Visibilitas = np.select(condition,value)
#####
#####

condition = [  training.Bangunan == "Bagus",
             training.Bangunan == "Sedang",
             
]

value = [2,1]

training.Bangunan = np.select(condition,value,0)


condition = [ 
            training.Peruntukan == "Pasar",
             training.Peruntukan == "Kantor",
             training.Peruntukan == "Ruko",
             training.Peruntukan == "Taman",
             training.Peruntukan == "Perumahan",
             training.Peruntukan == "Sawah",
             
]

value = [5,4,3,2,1,0]

training.Peruntukan = np.select(condition,value,0)

condition = [
    training.Bentuk_pemanfaatan == "Sewa"
]

value = [1]

training.Bentuk_pemanfaatan = np.select(condition,value,0)

In [4]:
condition = [  test.Pusat_kota == "Pusat",
             test.Pusat_kota == "Sedang",
              test.Pusat_kota == "Pinggir",
]

value = [3,2,1]

test.Pusat_kota = np.select(condition,value)
####
####

condition = [  test.Visibilitas == "Strategis",
             test.Visibilitas == "Sedang",
              test.Visibilitas == "Kurang",
]

value = [3,2,1]

test.Visibilitas = np.select(condition,value)
#####
#####

condition = [  test.Bangunan == "Bagus",
             test.Bangunan == "Sedang",
             
]

value = [2,1]

test.Bangunan = np.select(condition,value,0)


condition = [ 
            test.Peruntukan == "Pasar",
             test.Peruntukan == "Kantor",
             test.Peruntukan == "Ruko",
             test.Peruntukan == "Taman",
             test.Peruntukan == "Perumahan",
             test.Peruntukan == "Sawah",
             
]

value = [5,4,3,2,1,0]

test.Peruntukan = np.select(condition,value,0)

condition = [
    test.Bentuk_pemanfaatan == "Sewa"
]

value = [1]

test.Bentuk_pemanfaatan = np.select(condition,value,0)

In [5]:
training

,Bentuk_pemanfaatan,Peruntukan,Pusat_kota,Visibilitas,Bangunan,Luas
0,1,1,2,3,1,87580
1,1,1,2,3,1,87580
2,1,1,2,3,1,87580
3,1,1,2,3,1,87580
4,1,1,2,3,1,57945
...,...,...,...,...,...,...
123,0,2,3,3,0,1624
124,0,2,3,3,0,17033
125,0,2,3,3,0,2074
126,0,2,3,3,0,1700


In [6]:
test

,Bentuk_pemanfaatan,Peruntukan,Pusat_kota,Visibilitas,Bangunan,Luas
0,0,4,3,3,2,1192
1,0,4,3,2,2,997
2,1,5,3,3,2,14060
3,1,5,3,3,1,6025
4,0,0,1,1,0,10390
5,0,0,1,1,0,7745
6,1,0,1,1,0,5700
7,1,1,2,3,1,27024
8,0,1,1,2,1,8740
9,1,3,3,3,2,3614


# Dataframe to Dataset

In [7]:
def df_to_dataset(dataframe, shuffle=True, batch_size=4):
  dataframe = dataframe.copy()
  labels = dataframe.pop('Bentuk_pemanfaatan')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [8]:
train_ds = df_to_dataset(training)
val_ds = df_to_dataset(test)

2023-04-28 09:31:30.059600: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-04-28 09:31:30.059674: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-04-28 09:31:30.059720: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (gusanwa): /proc/driver/nvidia/version does not exist
2023-04-28 09:31:30.060316: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Feature Normalizer

In [9]:
def get_normalization_layer(name, dataset):
  # Create a Normalization layer for the feature.
  normalizer = layers.Normalization(axis=None)

  # Prepare a Dataset that only yields the feature.
  feature_ds = dataset.map(lambda x, y: x[name])

  # Learn the statistics of the data.
  normalizer.adapt(feature_ds)

  return normalizer

In [10]:
all_inputs = []
encoded_features = []

# Numerical features.
for header in ["Pusat_kota","Visibilitas","Bangunan","Luas","Peruntukan"]:
  numeric_col = tf.keras.Input(shape=(1,), name=header)
  normalization_layer = get_normalization_layer(header, train_ds)
  encoded_numeric_col = normalization_layer(numeric_col)
  all_inputs.append(numeric_col)
  encoded_features.append(encoded_numeric_col)

# Model build

In [11]:
all_features = tf.keras.layers.concatenate(encoded_features)
x = tf.keras.layers.Dense(32, activation="relu")(all_features)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(6)(x)

model = tf.keras.Model(all_inputs, output)

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=["accuracy"])

In [12]:
# Use `rankdir='LR'` to make the graph horizontal.
tf.keras.utils.plot_model(model, show_shapes=True, rankdir="LR")

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [13]:
model.fit(train_ds, epochs=100, validation_data=val_ds)

Epoch 1/100
32/32 [==============================] - 1s 12ms/step - loss: 1.7669 - accuracy: 0.2500 - val_loss: 1.5177 - val_accuracy: 0.3077
Epoch 2/100
32/32 [==============================] - 0s 4ms/step - loss: 1.4807 - accuracy: 0.4219 - val_loss: 1.3087 - val_accuracy: 0.4615
Epoch 3/100
32/32 [==============================] - 0s 5ms/step - loss: 1.3167 - accuracy: 0.5391 - val_loss: 1.1379 - val_accuracy: 0.7692
Epoch 4/100
32/32 [==============================] - 0s 5ms/step - loss: 1.1365 - accuracy: 0.6484 - val_loss: 1.0068 - val_accuracy: 0.7692
Epoch 5/100
32/32 [==============================] - 0s 5ms/step - loss: 1.0390 - accuracy: 0.7266 - val_loss: 0.9151 - val_accuracy: 0.7692
Epoch 6/100
32/32 [==============================] - 0s 5ms/step - loss: 0.9736 - accuracy: 0.6875 - val_loss: 0.8372 - val_accuracy: 0.7692
Epoch 7/100
32/32 [==============================] - 0s 5ms/step - loss: 0.8781 - accuracy: 0.7500 - val_loss: 0.7765 - val_accuracy: 0.7692
Epoch 8/100


Epoch 59/100
32/32 [==============================] - 0s 4ms/step - loss: 0.3677 - accuracy: 0.8672 - val_loss: 0.5188 - val_accuracy: 0.7692
Epoch 60/100
32/32 [==============================] - 0s 3ms/step - loss: 0.3734 - accuracy: 0.8906 - val_loss: 0.5170 - val_accuracy: 0.7692
Epoch 61/100
32/32 [==============================] - 0s 3ms/step - loss: 0.3661 - accuracy: 0.8828 - val_loss: 0.5154 - val_accuracy: 0.7692
Epoch 62/100
32/32 [==============================] - 0s 4ms/step - loss: 0.3558 - accuracy: 0.8906 - val_loss: 0.5181 - val_accuracy: 0.7692
Epoch 63/100
32/32 [==============================] - 0s 4ms/step - loss: 0.3741 - accuracy: 0.8672 - val_loss: 0.5203 - val_accuracy: 0.7692
Epoch 64/100
32/32 [==============================] - 0s 4ms/step - loss: 0.3505 - accuracy: 0.8672 - val_loss: 0.5135 - val_accuracy: 0.7692
Epoch 65/100
32/32 [==============================] - 0s 4ms/step - loss: 0.3847 - accuracy: 0.8750 - val_loss: 0.5063 - val_accuracy: 0.7692
Epoch 

In [14]:
model.predict(val_ds)

array([[ 1.5439024,  2.598068 , -6.514166 , -6.257872 , -5.4783673,
        -7.4534497],
       [ 5.107045 ,  2.908907 , -6.8929524, -5.658907 , -6.612163 ,
        -7.014919 ],
       [ 5.1678886,  2.9876623, -6.9872136, -5.7763467, -6.630157 ,
        -7.159117 ],
       [ 0.6001595,  2.8371897, -6.100852 , -6.3557854, -5.2292924,
        -6.919826 ],
       [ 0.976919 ,  3.7991123, -6.214421 , -6.633537 , -4.98209  ,
        -6.241874 ],
       [ 1.1530463,  3.4408135, -7.6669507, -6.80574  , -6.1852183,
        -8.140644 ],
       [ 5.1335754,  2.9432468, -6.934054 , -5.710115 , -6.6200094,
        -7.0777946],
       [ 0.7560632,  2.7429047, -6.023854 , -6.2924867, -5.163583 ,
        -6.914264 ],
       [ 3.5698802,  1.9094595, -4.6415496, -3.9040053, -4.1794057,
        -4.5123763],
       [ 2.554779 ,  1.8503424, -6.1863065, -5.421127 , -5.461224 ,
        -7.001449 ],
       [ 2.7405019,  1.7904652, -5.6272616, -5.6227674, -5.073341 ,
        -6.4962935],
       [ 3.8208172,  